<a href="https://colab.research.google.com/github/towet-dan/Django-signals/blob/main/Copy_of_2023_Malaria_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing the required libraries

In [ ]:
import tensorflow
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2


# the size of the image height and with

In [ ]:
image_size = [224, 224]

#Path for training, testing and validation data

In [ ]:
train_path = "/content/drive/MyDrive/DataSet/cell_images2/train"
test_path = "/content/drive/MyDrive/DataSet/cell_images2/test"
val_path = "/content/drive/MyDrive/DataSet/cell_images2/val"

#Instantiating the vgg19 class

In [ ]:
vgg19 = VGG19(input_shape = image_size + [3], weights='imagenet', include_top=False)

80134624/80134624 [==============================] - 0s 0us/step


#Here we are not going to train again the layers

In [ ]:
for layer in vgg19.layers:
  layer.trainable = False

In [ ]:
folders = os.listdir("/content/drive/MyDrive/DataSet/cell_images2/train")
print(folders)

['Parasitized', 'Uninfected']


#We are flattening the last layer

In [ ]:
x = Flatten()(vgg19.output)

#Creating the output using the dense layer, two output hence we use softmax activation

In [ ]:
predictions = Dense(len(folders), activation = "softmax")(x)

#Model instantiation

In [ ]:
model = Model(inputs = vgg19.input, outputs = predictions)

#Getting the summary of the model

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

#Here we are doing the data augumentation

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale= 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_set = train_datagen.flow_from_directory("/content/drive/MyDrive/DataSet/cell_images2/train", target_size = (224, 224), batch_size = 32, class_mode = "categorical")
test_set = test_datagen.flow_from_directory("/content/drive/MyDrive/DataSet/cell_images2/test", target_size = (224, 224), batch_size = 32, class_mode = "categorical")
val_set = val_datagen.flow_from_directory("/content/drive/MyDrive/DataSet/cell_images2/val", target_size = (224, 224), batch_size = 32, class_mode = "categorical")

Found 19311 images belonging to 2 classes.
Found 2761 images belonging to 2 classes.
Found 5516 images belonging to 2 classes.


#Model compilation process

In [ ]:
model.compile(optimizer="adam", loss = "categorical_crossentropy", metrics = ['accuracy'])

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

#Here is the training of the model

In [ ]:
history = model.fit(train_set, validation_data = val_set, epochs = 20, steps_per_epoch=len(train_set), validation_steps=len(val_set))

<ipython-input-23-e6aac5e51546>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_set, validation_data = val_set, epochs = 20, steps_per_epoch=len(train_set), validation_steps=len(val_set))


Epoch 1/20
604/604 [==============================] - 2795s 5s/step - loss: 0.2954 - accuracy: 0.8782 - val_loss: 0.2433 - val_accuracy: 0.9065
Epoch 2/20
604/604 [==============================] - 333s 551ms/step - loss: 0.2674 - accuracy: 0.8966 - val_loss: 0.2655 - val_accuracy: 0.8954
Epoch 3/20
604/604 [==============================] - 305s 504ms/step - loss: 0.2524 - accuracy: 0.9046 - val_loss: 0.3199 - val_accuracy: 0.8813
Epoch 4/20
604/604 [==============================] - 325s 538ms/step - loss: 0.2667 - accuracy: 0.9009 - val_loss: 0.2108 - val_accuracy: 0.9191
Epoch 5/20
604/604 [==============================] - 321s 532ms/step - loss: 0.2618 - accuracy: 0.9048 - val_loss: 0.2656 - val_accuracy: 0.8996
Epoch 6/20
604/604 [==============================] - 321s 532ms/step - loss: 0.2626 - accuracy: 0.9058 - val_loss: 0.2645 - val_accuracy: 0.9075
Epoch 7/20
604/604 [==============================] - 321s 531ms/step - loss: 0.2314 - accuracy: 0.9165 - val_loss: 0.2710 - v

In [ ]:
model.save('2023_mal_cell.h5')


#model evaluation

In [ ]:
model.evaluate(test_set, verbose=0)

[0.333173006772995, 0.894241213798523]

In [ ]:
score = model.evaluate(test_set, verdorse = 0)

NameError: ignored